# Simulation

Goal: to run a simulation based on **real-time location** tracking of ambulances. The simulation itself may or may not be used. But I hope that the **modules/ideas** making up this simulation could end up being useful.

- `Clocked`
- `Case events` are randomly entered

Personal note: [Github Markdown Including YouTube Videos](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet)

More information written below. 

# Taking in Cases:
Okay, so now that I have clock, I want to implement reading in the cases that happen in "real" time. 

### Easy but False Simulation 

- One way I could do that is to read it from a file. Characteristics of the events would be:

    - `GPS Location`
    - `Severity`
    - `Time it comes in`

- Characteristics of the case that are unpredictable, but useful:

    - The `time` it takes to accomplish the case
    - The actual `travel time`

### Harder but Realistic Simulation

- The other thing I could do is to have the events happen `randomly`. This is not hard. 

- The last thing I could do is to `manually input` the events which is the most difficult since this would be a continuous program that reacts based on user inputs. It would probably use GRPCs. 

- The random events can be shaped by seeds, which means we may be able to `shape the way new events are initialized`. 

# Implementing the Simulation

- Ambulances should be timed out. When they are sent on cases, they are timed in. There is a `time t` the ambulance is on-case. When the time finishes, it returns to Idle.
- New cases are started when the `arguments` come in and the time matches its start date. The other way to start cases is based on real time: manually input it. 
- We can randomly generate the `timeline` of cases that will backbone the simulation.

# Data Structures (ABC)


### Ambulances

a dict of tuple ()

```
(1) n <-- number of ambulances
(2) ambulance [n]
(3) each ambulance --> <status, time on-case>

```

### Base Locations

array 

```
(1) base location [x] where x <-- number of bases which is 100 

```

### Case Locations

array

```
(1) case location [y] where y <-- number of possible demand points 
    for which we have travel times. 

```

# Pseudocode Algorithm

I want to implement the following now:

```
for each time unit (t):
    
    update all times (t [amb1 .. amb_n])
    
    if a new case should start, then start it 
    
    if an on-case should end, then end it
    
    update the set coverage and coverage change (coverage, delta)
    

```

In [1]:
import time
import sys
import random
from enum import Enum

In [2]:
class Status(Enum):
    """ For ambulances """
    IDLE = 1
    ACTIVE = 2

In [3]:
class DataSource(Enum):
    """ There are at least three sources of events for the simulation. """
    RANDOM = 1

In [4]:
def generate_random_amb_event(amb_status, ambulance_set, demand_set):
    """ Given the set of ambulances and the set of demand points, randomly generate an event """
    """ The return type should specify ambulance and demand point. """
    
    chosen_amb = random.randint(0,len(amb_status)-1)
    
    return (chosen_amb, 0)

In [5]:
def get_next_event(amb_status, data_source, ambulance_set, demand_set, src_file):
    """ Reads the next event from data set or generates one. """
    
    if data_source is DataSource.RANDOM:
        (amb, demand) = generate_random_amb_event(amb_status, ambulance_set, demand_set)
    
    else: 
        raise Exception ("This should not happen.")
        
    return (amb, demand)

In [6]:
def change_state(amb_status, chosen_amb):
    """ Takes the ambulance status dictionary, and an event. Toggles ambulance setting """    
    
    if type(amb_status) is not dict: raise Exception("Wrong type. ") 
        
    
    for each in amb_status:
        (status, time) = amb_status[each]
        if time > -1: amb_status[each] = (status, time + 1)
        if amb_status[each][1] >= 1200:
            amb_status[each] = (Status.IDLE, -1)
    
    if chosen_amb < 0: 
        return
    
    # TODO This line of code is not supposed to be here. 
    if amb_status[chosen_amb][1] == -1:
        amb_status[chosen_amb] = (Status.ACTIVE, 0)
        
    

In [9]:
def execute_simulation(run_time=1, speed_times=4, amb_status = None, 
                  data_source = None, ambulance_set = None, demand_set = None):
    """ Starts a clock """
    """ Every unit of time, update the next ambulance event, print, update time. """
    """ Each ambulance should also know how long it's gone for. It should keep track of itself. """
    
    if amb_status == None: raise Exception ("param amb_status is None")
    if type(amb_status) is not dict: raise Exception ("amb_status wrong type")
    if type(run_time) is not int: raise Exception("run_time not int")
    if type(speed_times) is not int: raise Exception("speed_times not int")

    start = time.time()
    time_diff = 0

    while True:
        (chosen_amb, demand_location) = (-1, -1)
        time.sleep(1/speed_times)
        
        if time_diff % 60 == 0:
            print ("Time: %3i seconds" %(time_diff))
        
        
        
        # Read the next event and then change the state:
        if time_diff % 300 == 0:
            (chosen_amb, demand_location) = get_next_event(
                amb_status, data_source, ambulance_set, demand_set, None)

        change_state(amb_status, chosen_amb)
        
        
        
        if time_diff % 60 == 0:
            print("The amb-event is " , (chosen_amb, demand_location))
            for ID in amb_status: 
                print("Ambulance %2i\t" %(ID), amb_status[ID])        
        
            print()
        now = time.time()
        time_diff = int((now - start) * speed_times)

        if time_diff > run_time: break 
            
        

In [ ]:
# The following two variables will change the speed and amount of time the simulation is run.

# 2 hours
run_time = 7200 # in simulated seconds

speed = 50  # in (simulated seconds * speed) real seconds

# It's still possible to change the granularity of this unit of time. 1/10 of a second granularity could be better.

ambulance_count = 10
amb_status = {} 

# Each ambulance maps to a tuple (status, time active)
for ID in range(0, ambulance_count):
    amb_status[ID] = (Status.IDLE, -1)

execute_simulation(run_time=run_time, speed_times=speed, amb_status = amb_status, data_source = DataSource.RANDOM)

Time:   0 seconds
The amb-event is  (3, 0)
Ambulance  0	 (<Status.IDLE: 1>, -1)
Ambulance  1	 (<Status.IDLE: 1>, -1)
Ambulance  2	 (<Status.IDLE: 1>, -1)
Ambulance  3	 (<Status.ACTIVE: 2>, 0)
Ambulance  4	 (<Status.IDLE: 1>, -1)
Ambulance  5	 (<Status.IDLE: 1>, -1)
Ambulance  6	 (<Status.IDLE: 1>, -1)
Ambulance  7	 (<Status.IDLE: 1>, -1)
Ambulance  8	 (<Status.IDLE: 1>, -1)
Ambulance  9	 (<Status.IDLE: 1>, -1)

Time:  60 seconds
The amb-event is  (-1, -1)
Ambulance  0	 (<Status.IDLE: 1>, -1)
Ambulance  1	 (<Status.IDLE: 1>, -1)
Ambulance  2	 (<Status.IDLE: 1>, -1)
Ambulance  3	 (<Status.ACTIVE: 2>, 52)
Ambulance  4	 (<Status.IDLE: 1>, -1)
Ambulance  5	 (<Status.IDLE: 1>, -1)
Ambulance  6	 (<Status.IDLE: 1>, -1)
Ambulance  7	 (<Status.IDLE: 1>, -1)
Ambulance  8	 (<Status.IDLE: 1>, -1)
Ambulance  9	 (<Status.IDLE: 1>, -1)

Time: 120 seconds
The amb-event is  (-1, -1)
Ambulance  0	 (<Status.IDLE: 1>, -1)
Ambulance  1	 (<Status.IDLE: 1>, -1)
Ambulance  2	 (<Status.IDLE: 1>, -1)
Ambulance  

There is a bug with randomly generating events. If I choose an on-duty ambulance, the case is not well-defined. This program should really be running on terminal.

The unit of time should be seconds, since that is what the data uses and it seems granular yet efficient enough.